In [62]:
import time
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner
from sagemaker.predictor import RealTimePredictor
from sagemaker.model import Model
import json
import numpy as np

In [2]:
# Number of output classes
num_classes = 2

# Number of training sample in training set
num_training_samples = 208

# Number of layers for neural network
num_layers = 18

# Image augmentation
augmentation_type = "crop_color_transform"

# Batch size for training
mini_batch_size = 64

# Input image shape
image_shape = '3,50,50'

epochs = 50

learning_rate = 0.001

use_pretrained_model = 1

In [3]:
job_name_prefix = "food-zone"
timestamp = time.strftime('-%Y-%m-%d-%H-%S', time.gmtime())
job_name = job_name_prefix + timestamp
print(job_name)

food-zone-2020-03-10-07-18


In [4]:
bucket = 'food-zone'
input_prefix = 'input/recordio'
input_train = 's3://{}/{}/train'.format(bucket, input_prefix)
input_test = 's3://{}/{}/test'.format(bucket, input_prefix)

In [5]:
output_prefix = "/output"
output_path = 's3://{}/{}'.format(bucket, output_prefix)

In [6]:
instance_count = 1
instance_type = 'ml.p2.xlarge'
volume_size_gb = 5

In [7]:
role = get_execution_role()
training_image = get_image_uri(boto3.Session().region_name, 'image-classification')

In [8]:
train_timeout = 3600

In [9]:
session = sagemaker.Session()
estimator = sagemaker.estimator.Estimator(training_image,
                                         role,
                                         train_instance_count=instance_count,
                                         train_instance_type=instance_type,
                                         train_volume_size=volume_size_gb,
                                         train_max_run=train_timeout,
                                         output_path=output_path,
                                         sagemaker_session=session,
                                         input_mode='Pipe')

In [10]:
estimator.set_hyperparameters(num_classes=num_classes,
                             num_training_samples=num_training_samples,
                             num_layers=num_layers,
                             mini_batch_size=mini_batch_size,
                             augmentation_type=augmentation_type,
                             epochs=epochs,
                             image_shape=image_shape,
                             learning_rate=learning_rate,
                             use_pretrained_model=use_pretrained_model)

In [11]:
s3_input_train = sagemaker.s3_input(s3_data=input_train, content_type='application/x-recordio')
s3_input_validation = sagemaker.s3_input(s3_data=input_test, content_type='application/x-recordio')

In [110]:
estimator.fit({
    'train': s3_input_train,
    'validation': s3_input_validation}, 
    job_name=job_name)

2020-03-09 19:48:34 Starting - Starting the training job...
2020-03-09 19:48:36 Starting - Launching requested ML instances......
2020-03-09 19:49:42 Starting - Preparing the instances for training.........
2020-03-09 19:51:18 Downloading - Downloading input data...
2020-03-09 19:51:56 Training - Downloading the training image......
2020-03-09 19:53:01 Training - Training image download completed. Training in progress..Docker entrypoint called with argument(s): train
[03/09/2020 19:53:05 INFO 139733361231680] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/image_classification/default-input.json: {u'beta_1': 0.9, u'gamma': 0.9, u'beta_2': 0.999, u'optimizer': u'sgd', u'use_pretrained_model': 0, u'eps': 1e-08, u'epochs': 30, u'lr_scheduler_factor': 0.1, u'num_layers': 152, u'image_shape': u'3,224,224', u'precision_dtype': u'float32', u'mini_batch_size': 32, u'weight_decay': 0.0001, u'learning_rate': 0.1, u'momentum': 0}
[03/09/2020 19:53:05 INFO 13973336123168

[03/09/2020 19:54:03 INFO 139733361231680] Epoch[15] Train-accuracy=0.640625
[03/09/2020 19:54:03 INFO 139733361231680] Epoch[15] Time cost=2.390
[03/09/2020 19:54:04 INFO 139733361231680] Epoch[15] Validation-accuracy=0.421875
[03/09/2020 19:54:07 INFO 139733361231680] Epoch[16] Train-accuracy=0.671875
[03/09/2020 19:54:07 INFO 139733361231680] Epoch[16] Time cost=2.347
[03/09/2020 19:54:07 INFO 139733361231680] Epoch[16] Validation-accuracy=0.414062
[03/09/2020 19:54:10 INFO 139733361231680] Epoch[17] Train-accuracy=0.682292
[03/09/2020 19:54:10 INFO 139733361231680] Epoch[17] Time cost=2.434
[03/09/2020 19:54:10 INFO 139733361231680] Epoch[17] Validation-accuracy=0.429688
[03/09/2020 19:54:13 INFO 139733361231680] Epoch[18] Train-accuracy=0.651042
[03/09/2020 19:54:13 INFO 139733361231680] Epoch[18] Time cost=2.344
[03/09/2020 19:54:13 INFO 139733361231680] Epoch[18] Validation-accuracy=0.437500
[03/09/2020 19:54:16 INFO 139733361231680] Epoch[19] Train-accuracy=0.619792
[03/09/2020


2020-03-09 19:56:06 Uploading - Uploading generated training model
2020-03-09 19:56:06 Completed - Training job completed
Training seconds: 288
Billable seconds: 288


# Create a tuning job

In [31]:
hyperparameter_ranges = {
    'learning_rate': ContinuousParameter(0.001, 1.0),
    'mini_batch_size': IntegerParameter(16,64),
    'optimizer': CategoricalParameter(['sgd','adam'])
}

objective_metric_name = 'validation:accuracy'
objective_type = 'Maximize'
max_jobs = 2
max_parallel_jobs = 1

In [32]:
job_name_prefix = 'tune-job'
timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
job_name = job_name_prefix + timestamp
print(job_name)

tune-job-2020-03-10-08-13-25


In [33]:
tuner = HyperparameterTuner(estimator=estimator,
                          objective_metric_name=objective_metric_name,
                          hyperparameter_ranges=hyperparameter_ranges,
                          objective_type=objective_type,
                          max_jobs=max_jobs,
                          max_parallel_jobs=max_parallel_jobs)

In [34]:
tuner.fit({
    'train': s3_input_train,
    'validation': s3_input_validation,
}, job_name=job_name)

tuner.wait()

................................................................................................................................................................................................................!


# Deploy model

In [36]:
role = get_execution_role()
hosting_image = get_image_uri(boto3.Session().region_name, 'image-classification')

In [39]:
instance_count = 1
instance_type = 'ml.m4.xlarge'

In [53]:
model_name_prefix = 'food-zone-model'
timestamp = time.strftime('-%Y-%m-%d-%H-%M-%s', time.gmtime())
model_name = model_name_prefix + timestamp
print(model_name)

food-zone-model-2020-03-10-09-36-1583832972


In [57]:
model_path = 's3://food-zone//output/tune-job-2020-03-10-08-13-25-001-0aa7d686/output/model.tar.gz'
model = Model(
    name=model_name,
    model_data = model_path,
    image=hosting_image,
    role=role,
    predictor_cls=lambda endpoint_name, sagemaker_session: RealTimePredictor(endpoint_name, sagemaker_session)
)

In [58]:
endpoint_name_prefix = 'food-zone-model'
timestamp = time.strftime('-%Y-%m-%d-%H-%M-%s', time.gmtime())
endpoint_name = endpoint_name_prefix + timestamp
print(endpoint_name)

food-zone-model-2020-03-10-09-36-1583833018


In [59]:
predictor = model.deploy(
    endpoint_name=endpoint_name,
    initial_instance_count=instance_count,
    instance_type=instance_type)

-----------------!

In [64]:
def predict_food(image_file):
    with open(image_file, 'rb') as f:
        payload = f.read()
        payload = bytearray(payload)
        
    response = predictor.predict(payload)
    result = json.loads(response)
    
    print(result)
    
    predicted_class = np.argmax(result)
    if predicted_class == 0:
        print('enchiladas')
    else:
        print('taco')

In [65]:
predict_food('images/enchiladas/1382541970364.jpeg')

[0.4550020396709442, 0.5449979305267334]
taco


In [66]:
predict_food('images/taco/39654.jpg')

[0.4618596136569977, 0.5381404161453247]
taco


In [69]:
sagemaker.Session().delete_endpoint(endpoint_name)

ClientError: An error occurred (ValidationException) when calling the DeleteEndpoint operation: Could not find endpoint "arn:aws:sagemaker:us-east-1:756448110530:endpoint/food-zone-model-2020-03-10-09-36-1583833018".

In [70]:
sagemaker.Session().delete_model(model_name)